In [2]:
import requests
import json

# Replace these with your Redash instance details
REDASH_HOST = "https://redash.truckitin.ai"
API_KEY = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"

# URL for the data sources API
url = f"{REDASH_HOST}/api/data_sources"

# Headers with the API key
headers = {"Authorization": f"Key {API_KEY}"}

# Make the request
response = requests.get(url, headers=headers)

# Check for a successful response
if response.status_code == 200:
    # Parse the JSON response
    data_sources = response.json()
    print(json.dumps(data_sources, indent=2))
else:
    print(f"Failed to get data sources: {response.status_code} - {response.text}")

[
  {
    "name": "Amigu-Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 18
  },
  {
    "name": "Billing Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 8
  },
  {
    "name": "Datawarehouse",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 6
  },
  {
    "name": "Dimagh Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 2
  },
  {
    "name": "Ledger Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 4
  },
  {
    "name": "rider_db_notification",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "mysql",
    "id": 15
  }

In [3]:
import requests
import time
import pandas as pd


def run_query(api_key, query, data_source_id):
    base_url = "https://redash.truckitin.ai/api"
    headers = {"Authorization": f"Key {api_key}"}

    # Create and execute the query
    response = requests.post(
        f"{base_url}/queries",
        headers=headers,
        json={"query": query, "data_source_id": data_source_id},
    )
    query_id = response.json()["id"]

    # Execute the query
    response = requests.post(f"{base_url}/queries/{query_id}/results", headers=headers)
    job = response.json()["job"]

    # Poll for query results
    while job["status"] not in (3, 4):  # 3: completed, 4: failed
        response = requests.get(f'{base_url}/jobs/{job["id"]}', headers=headers)
        job = response.json()["job"]
        time.sleep(1)

    if job["status"] == 3:
        results = requests.get(
            f"{base_url}/queries/{query_id}/results.json", headers=headers
        ).json()["query_result"]["data"]
        return pd.DataFrame(results["rows"])
    else:
        raise Exception("Query execution failed.")


# Usage example
api_key = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"
data_source_id = 14  # You need to know this beforehand
query = "SELECT * FROM OrderDetails LIMIT 10"

df = run_query(api_key, query, data_source_id)
print(df)

KeyError: 'id'

In [9]:
import requests
import json
import pandas as pd
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

REDASH_HOST = "https://redash.truckitin.ai"
API_KEY = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"
DATA_SOURCE_ID = "14"

# Ad-hoc SQL query
query = """
    SELECT id,
       consignment_id,
       origin_city_id,
       origin_city_name,
       city_id,
       dest_city_name,
       warehouse_id,
       warehouse_title,
       area_id,
       area_title,
       sort_addr_id,
       sort_addr_title,
       Concat(area_title, ' > ', sort_addr_title) AS L3_L4
    FROM   orderdetails
    LIMIT 10"""

# URL for running the query
url = f"{REDASH_HOST}/api/query_results"

# Headers with the API key
headers = {"Authorization": f"Key {API_KEY}", "Content-Type": "application/json"}

# Payload with the data source ID and query
payload = {"data_source_id": DATA_SOURCE_ID, "query": query}


def run_query(url, headers, payload):
    try:
        # Make the request
        response = requests.post(url, headers=headers, data=json.dumps(payload))

        # Check for a successful response
        response.raise_for_status()

        # Parse the JSON response
        result = response.json()

        print(json.dumps(result, indent=2))

        # Extract the query results
        query_results = result["query_result"]["data"]["rows"]

        # Convert the query results to a Pandas DataFrame
        df = pd.DataFrame(query_results)

        return df

    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return None
    except KeyError as e:
        logging.error(f"Unexpected response format: {e}")
        return None


# Run the query and get the DataFrame
df = run_query(url, headers, payload)

if df is not None:
    logging.info("Query executed successfully.")
    # Proceed with further processing on df
    print(df.head())
else:
    logging.error("Query execution failed.")

ERROR:root:Unexpected response format: 'query_result'
ERROR:root:Query execution failed.


{
  "job": {
    "status": 1,
    "error": "",
    "id": "8e3aac56-161e-457b-9ab2-35dd85ad6d34",
    "query_result_id": null,
    "updated_at": 0
  }
}


In [14]:
import requests
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

REDASH_HOST = "https://redash.truckitin.ai"
API_KEY = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"
DATA_SOURCE_ID = "14"

query = """
    SELECT id,
           consignment_id,
           origin_city_id,
           origin_city_name,
           city_id,
           dest_city_name,
           warehouse_id,
           warehouse_title,
           area_id,
           area_title,
           sort_addr_id,
           sort_addr_title,
           CONCAT(area_title, ' > ', sort_addr_title) AS L3_L4
    FROM OrderDetails
    LIMIT 10"""

headers = {"Authorization": f"Key {API_KEY}", "Content-Type": "application/json"}


def submit_query():
    url = f"{REDASH_HOST}/api/query_results"
    payload = {"data_source_id": DATA_SOURCE_ID, "query": query, "max_age": 0}
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["job"]["id"]


def check_job_status(job_id):
    url = f"{REDASH_HOST}/api/jobs/{job_id}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()["job"]


def get_query_results(query_result_id):
    url = f"{REDASH_HOST}/api/query_results/{query_result_id}.json"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()["query_result"]["data"]["rows"]


def run_query():
    try:
        job_id = submit_query()
        logging.info(f"Job ID: {job_id}")

        while True:
            job = check_job_status(job_id)
            status = job["status"]
            logging.info(f"Job Status: {status}")

            if status == 3:  # completed
                return get_query_results(job["query_result_id"])
            elif status == 4:  # failed
                logging.error(
                    f"Query execution failed: {job.get('error', 'No error message provided')}"
                )
                return None

            # Use ThreadPoolExecutor for non-blocking wait
            with ThreadPoolExecutor() as executor:
                future = executor.submit(check_job_status, job_id)
                job = future.result(timeout=1)

    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
    except KeyError as e:
        logging.error(f"Unexpected response format: {e}")
    return None


# Run the query and get the DataFrame
results = run_query()
if results:
    df = pd.DataFrame(results)
    logging.info("Query executed successfully.")
    print(df.head())
else:
    logging.error("Query execution failed.")

   area_id sort_addr_title consignment_id  city_id origin_city_name  \
0     6209            None      WR4025670      772           Lahore   
1     2012            None      WR4025672        1          Karachi   
2     6209            None      WR4025673      772       Faisalabad   
3      364            None      WR4025674        2           Lahore   
4      561            None      WR4025675        3           Lahore   

   warehouse_id dest_city_name  origin_city_id   area_title L3_L4  \
0            55        PATOKI                2      Pattoki  None   
1             2        Karachi               1      Block 1  None   
2            55        PATOKI               10      Pattoki  None   
3             3         Lahore               2  Bahria Town  None   
4             4      Hyderabad               2        Other  None   

   warehouse_title  id sort_addr_id  
0  PTI Head Office   1         None  
1     LuckyOne Hub   2         None  
2  PTI Head Office   3         None  
3  LHE